In [17]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_excel('UpdatedAgainTenders.xlsx')

df['Start Date'] = pd.to_datetime(df['Start Date'])

df.sort_values(by='Start Date', inplace=True)

# dictionary to store groups for each tender
tender_groups = {}

# Initialize a group counter
group_counter = 1

# Iterate through the DataFrame and assign groups based on the 90-day range
for index, row in df.iterrows():
    if pd.notna(row['Start Date']):
        # Initialize a list to store groups for the current tender
        groups = []
        
        # Iterate through previous rows to find groups
        for i in range(index - 1, -1, -1):
            prev_row = df.iloc[i]
            time_diff = row['Start Date'] - prev_row['Start Date']
            
            if time_diff <= timedelta(days=90):
                groups.append(prev_row['Reference Number'])
            else:
                break
        
        # Add the current tender to its own group
        groups.append(row['Reference Number'])
        
        # Update the dictionary with the groups for the current tender
        tender_groups[row['Reference Number']] = groups
    else:
        # Assign 'Group 0' to tenders with null start dates
        tender_groups[row['Reference Number']] = ['Group 0']

# Create a new column 'Group' in the DataFrame
#df['Group'] = df['Reference Number'].map(lambda x: ', '.join(tender_groups.get(x, [])))
#code is updated to avoid the below type error 
df['Group'] = df['Reference Number'].map(lambda x: ', '.join(map(str, tender_groups.get(x, []))))


df.reset_index(drop=True, inplace=True)

# Save the DataFrame back to an Excel file if needed
df.to_excel('output_file.xlsx', index=False)


TypeError: sequence item 1524: expected str instance, int found